<a href="https://colab.research.google.com/github/pablobots/private_test/blob/main/Pediatric_Bone_Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/pablobots/private_test/blob/main/Pediatric_Bone_Age.ipynb


In [8]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


In [1]:
!pip install -q kaggle

In [2]:
!mkdir ~/.kaggle 

In [3]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [36]:
from google.colab import drive
drive.flush_and_unmount()

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets list

ref                                                           title                                                size  lastUpdated          downloadCount  
------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
utkarshxy/who-worldhealth-statistics-2020-complete            World Health Statistics 2020|Complete|Geo-Analysis    1MB  2021-01-10 18:25:49           2884  
gpreda/pfizer-vaccine-tweets                                  Pfizer Vaccine Tweets                               541KB  2021-01-13 08:04:33           1775  
arashnic/covid19-case-surveillance-public-use-dataset         Covid-19 Case Surveillance Public Use Dataset        46MB  2020-12-21 02:24:21           1445  
ashkhagan/women-representation-in-city-property-sanfrancisco  Women Representation in City Property SanFrancisco    3KB  2020-12-13 05:18:14            431  
google/android-smartphones-high-accuracy-datasets   

In [6]:
!kaggle datasets download -d kmader/rsna-bone-age -p ./

100% 9.28G/9.29G [03:38<00:00, 27.2MB/s]
100% 9.29G/9.29G [03:38<00:00, 45.6MB/s]


In [7]:
!unzip rsna-bone-age.zip

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  inflating: boneage-training-dataset/boneage-training-dataset/4264.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4265.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4266.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4268.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4269.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4270.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4271.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4272.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4273.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4275.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4276.png  
  inflating: boneage-training-dataset/boneage-training-dataset/4277.png  
  inflating: boneage-training-dataset/boneage

In [35]:
!rm rsna-bone-age.zip
!rm -r boneage-test-dataset
!rm boneage-test-dataset.csv

rm: cannot remove 'boneage-test-dataset': No such file or directory
rm: cannot remove 'boneage-test-dataset.csv': No such file or directory


In [48]:
!rm -r boneage-dataset-deit
!mkdir boneage-dataset-deit
!mkdir boneage-dataset-deit/train
!mkdir boneage-dataset-deit/val

In [ ]:
import pandas as pd
import os
import shutil
import random

input_dir = "/content/boneage-training-dataset/boneage-training-dataset"
output_dir_train = "boneage-dataset-deit/train"
output_dir_val = "boneage-dataset-deit/val"
df = pd.read_csv('boneage-training-dataset.csv')

for index, row in df.iterrows():
  print(row['id'], row['boneage'], row['male'])
  
  file_path = os.path.join(input_dir, str(row['id']) + ".png")
  boneage = str(row['boneage'])

  output_subdir_train = os.path.join(output_dir_train, boneage)
  if not os.path.exists(output_subdir_train):
    os.mkdir(output_subdir_train)
  output_subdir_val = os.path.join(output_dir_val, boneage)
  if not os.path.exists(output_subdir_val):
    os.mkdir(output_subdir_val)

  if random.uniform(0, 1) <= 0.7:
    shutil.copy(file_path, output_subdir_train)
  else:
    shutil.copy(file_path, output_subdir_val)

In [54]:
!git clone https://github.com/facebookresearch/deit.git

Cloning into 'deit'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 98 (delta 2), reused 8 (delta 2), pack-reused 89
Unpacking objects: 100% (98/98), done.


In [55]:
%cd deit/

/content/deit


In [51]:
!pip install torch torchvision

In [52]:
!pip install timm==0.3.2

     |████████████████████████████████| 245kB 8.8MB/s 


In [ ]:
!python main.py --model deit_small_patch16_224 --batch-size 32 --epochs 400 --output_dir /content/deit/output --data-path /content/boneage-dataset-deit

Not using distributed mode
Namespace(aa='rand-m9-mstd0.5-inc1', batch_size=32, clip_grad=None, color_jitter=0.4, cooldown_epochs=10, cutmix=1.0, cutmix_minmax=None, data_path='/content/boneage-dataset-deit', data_set='IMNET', decay_epochs=30, decay_rate=0.1, device='cuda', dist_eval=False, dist_url='env://', distillation_alpha=0.5, distillation_tau=1.0, distillation_type='none', distributed=False, drop=0.0, drop_path=0.1, epochs=400, eval=False, inat_category='name', input_size=224, lr=0.0005, lr_noise=None, lr_noise_pct=0.67, lr_noise_std=1.0, min_lr=1e-05, mixup=0.8, mixup_mode='batch', mixup_prob=1.0, mixup_switch_prob=0.5, model='deit_small_patch16_224', model_ema=True, model_ema_decay=0.99996, model_ema_force_cpu=False, momentum=0.9, num_workers=10, opt='adamw', opt_betas=None, opt_eps=1e-08, output_dir='/content/deit/output', patience_epochs=10, pin_mem=True, recount=1, remode='pixel', repeated_aug=True, reprob=0.25, resplit=False, resume='', sched='cosine', seed=0, smoothing=0.1

In [ ]:
!python main.py --eval --resume /content/deit/output/checkpoint.pth --model deit_small_patch16_224 --batch-size 32 --data-path /content/boneage-dataset-deit

In [ ]:
from PIL import Image
import torch
import timm

import torchvision.transforms as transforms

from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

model = torch.hub.load('facebookresearch/deit:main', 'deit_small_patch16_224', pretrained=False)
ckpt_path = '/content/deit/output/checkpoint.pth'

ckpt_dict = torch.load(ckpt_path)
model.load_state_dict(ckpt_dict["model"])

model.eval()

transform = transforms.Compose([
    transforms.Resize(256, interpolation=3),  # to maintain same ratio w.r.t. 224 images
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
])

img_path = '/content/dataset_3/test/capsella_bursa_pastoris/159.png'
#img_path = '/content/dataset_3/test/scentless_mayweed/10.png'
#img_path = '/content/dataset_3/test/small_flowered_cranesbill/1.png'
img = Image.open(img_path).convert("RGB")
img = transform(img)[None,]

out = model(img)
pred_label = torch.argmax(out)
print(pred_label.item())


In [ ]:
!git clone https://github.com/facebookresearch/deit.git

In [ ]:
%cd deit/